In [ ]:
from mosqito.sq_metrics import sharpness_din_tv, loudness_zwtv, roughness_dw, pr_ecma_perseg, tnr_ecma_perseg
import numpy as np
import soundfile as sf

#### Calculate calibration factor

In [56]:
P_REF = 20e-6 

def sound_pressure(audio_data):
    p_rms = np.sqrt(np.mean(audio_data**2))  # RMS berechnen
    laeq = 20 * np.log10(p_rms / P_REF)  # Umrechnung in dB SPL
    return laeq

def calibration_value(audio_data, expected_l_ch1, expected_l_ch2):
    raw_laeg_ch1 = sound_pressure(audio_data[:, 0])
    raw_laeg_ch2 = sound_pressure(audio_data[:, 1])

    cal_ch1 = expected_l_ch1 - raw_laeg_ch1
    cal_ch2 = expected_l_ch2 - raw_laeg_ch2

    return cal_ch1, cal_ch2

def apply_calibration(audio_data, cal_ch1, cal_ch2):
    audio_calibrated = np.zeros_like(audio_data)
    audio_calibrated[:, 0] = audio_data[:, 0] * (10**(cal_ch1 / 20))  # dB-Wert in linear gain Faktor und multipliziern
    audio_calibrated[:, 1] = audio_data[:, 1] * (10**(cal_ch2 / 20)) 
    return audio_calibrated

# Audiosignal laden
file_path = "../dataset/audiofiles/1132730_16.wav"
audio_data, fs = sf.read(file_path)

# kallibrieren
cal_ch1, cal_ch2 = calibration_value(audio_data, 64.13, 64.57)


#### Compute acoustic features

In [ ]:
# load dataset
import pandas as pd

path = '../dataset/datasetSmall.csv'
df = pd.read_csv(path)

if 'temporal_audio_features' not in df.columns:
    df['temporal_audio_features'] = None

In [ ]:
# compute audio features and save to dataset
import os
import soundfile as sf

def compute_acoustic_features(audio, sr):

    _, N_time, _, _, _ = loudness_zwtv(audio, sr) # N_time
    S, _ = sharpness_din_tv(audio, fs=sr) # S
    _, R_time, _, _, _ = roughness_dw(audio, sr) # R_time
    t_pr, _, _, _, _ = pr_ecma_perseg(audio, sr) # t_pr
    t_tnr, _, _, _, _ = tnr_ecma_perseg(audio, sr) # t_tnr
    
    return N_time, S, R_time, t_pr, t_tnr

def scale_acoustic_features_to_audio_length(feature_array, audio_length):

    original_length = len(feature_array)
    block_size = original_length / audio_length
    result = np.zeros(audio_length)
    
    for i in range(audio_length):
        start_idx = int(i * block_size)
        end_idx = int(min((i + 1) * block_size, original_length))
        
        # Average the values in this block
        if start_idx < end_idx:
            result[i] = np.mean(feature_array[start_idx:end_idx])
        else:
            result[i] = feature_array[start_idx] if start_idx < original_length else 0
            
    return result

folder_path = '../dataset/test_audiofiles'
audio_files = [f for f in os.listdir(folder_path) if f.endswith(('.wav'))]
    
    # Process each audio file
for audio_file in audio_files:
    file_path = os.path.join(folder_path, audio_file)
    
    try:
        # Load audio
        audio_data, sr = sf.read(file_path)

        # calibrate audio and convert to mono
        calibrated_audio = apply_calibration(audio_data, cal_ch1, cal_ch2)
        mono_audio = np.mean(calibrated_audio, axis=1)

        # compute acoustic features
        loudness, sharpness, roughness, prominence_ratio, tone_to_noise_ratio = compute_acoustic_features(mono_audio, sr)

        # averagte features to audio length
        time_scaling_factor = 1
        length_audio = np.floor(len(mono_audio)/sr) + 1
        loudness = scale_acoustic_features_to_audio_length(loudness, int((length_audio*time_scaling_factor)))
        sharpness = scale_acoustic_features_to_audio_length(sharpness, int((length_audio*time_scaling_factor)))
        roughness = scale_acoustic_features_to_audio_length(roughness, int((length_audio*time_scaling_factor)))

        audio_features = {
            "loudness": loudness.tolist(),
            "sharpness": sharpness.tolist(),
            "roughness": roughness.tolist(),
            "prominence_ratio": prominence_ratio.tolist(),
            "tone_to_noise_ratio": tone_to_noise_ratio.tolist()
        }

        # push to dataset
        file_name = file_path.split('/')[-1].split('.')[0]
        row_idx = df[df['file_name'] == file_name].index.to_list()

        if row_idx:
            df.at[row_idx[0], 'temporal_audio_features'] = audio_features
            print('\n done processing file ', audio_file)
            print('\n')
        else:
            print(f"No row found with file_name: {file_name}")
    except Exception as e:
        print(f"Error processing {audio_file}: {str(e)}")

df.to_csv('../dataset/FinalDatasetWithAcousticTemporalFeatures.csv', sep=';', index=False)

[Warning] Signal resampled to 48 kHz fulfill the standard requirements and allow calculation.
[Warning] Signal resampled to 48 kHz to allow calculation. To fulfill the standard requirements fs should be >=48 kHz.
[Warning] when computing sharpness from time-varying loudness, a transient effect appears on the first points. To cut it, use 'skip='
[Warning] Signal resampled to 48 kHz fulfill the standard requirements and allow calculation.


/Users/matthiaserdmann/Documents/SoftwareDev/WebDev/soundscape-search/.venv/lib/python3.11/site-packages/mosqito/sq_metrics/roughness/roughness_ecma/_estimate_fund_mod_rate.py:43: RuntimeWarning: divide by zero encountered in divide
  crit = np.abs( (f_p_temp[ic] / (R_i0[ic] * f_p[i0])) - 1)



 done processing file  1132730_32.wav


[Warning] Signal resampled to 48 kHz fulfill the standard requirements and allow calculation.
[Warning] Signal resampled to 48 kHz to allow calculation. To fulfill the standard requirements fs should be >=48 kHz.
[Warning] when computing sharpness from time-varying loudness, a transient effect appears on the first points. To cut it, use 'skip='
[Warning] Signal resampled to 48 kHz fulfill the standard requirements and allow calculation.

 done processing file  1134651_42.wav


[Warning] Signal resampled to 48 kHz fulfill the standard requirements and allow calculation.
[Warning] Signal resampled to 48 kHz to allow calculation. To fulfill the standard requirements fs should be >=48 kHz.
[Warning] when computing sharpness from time-varying loudness, a transient effect appears on the first points. To cut it, use 'skip='
[Warning] Signal resampled to 48 kHz fulfill the standard requirements and allow calculation.

 done processing file  1996384_28.w

In [ ]:
# store to database
from pymongo import MongoClient

client = MongoClient('mongodb://localhost:27017/')
db = client['soundscape_search']
# collection = db['dataset_demo_temporalFeatures']
collection = db['test_dataset']

for index, soundscape in df.iterrows():
    collection.insert_one(soundscape.to_dict())